In [ ]:
import nltk
import re
import string
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Importação do arquivo

In [ ]:
#Ajustar tamanho máximo de linha visível. Max caracteres por tweet: 280
pd.set_option('max_colwidth', 300)

In [ ]:
#Importar dados do dataset encoding incompatível com utf-8
data = pd.read_csv("train.csv", encoding='ISO-8859-1')
#Remover colunas desncessárias
data = data.drop(columns=['ItemID','Sentiment'])

In [ ]:
#Criar coluna com o tamanho de caracteres de cada tweet
data['length'] = [len(tweet[0]) for tweet in data.values]

In [ ]:
#Linhas com mais e menos caracteres
data.sort_values(by='length', ascending=False)

In [ ]:
#Dados sobre a quantidade de caracteres
data.describe()

## Contagem de emoticons
fonte: https://www.kaggle.com/youben/twitter-sentiment-analysis

In [ ]:
#Contagem (com alterações)
tweets_text = data.SentimentText.str.cat()
emos = set(re.findall(r" ([xX:;][-']?\S) ",tweets_text))
emos_count = []
for emo in emos:
    emos_count.append((tweets_text.count(emo), emo))
sorted(emos_count,reverse=True)

In [ ]:
#Busca (com alterações)
HAPPY_EMO = r" ([xX;:]-?[bdD)]|:-?[\)]|[;:][pP]) "
SAD_EMO = r" (:'?[xXsS/|\(]) "
print("Happy emoticons:", set(re.findall(HAPPY_EMO, tweets_text)))
print("Sad emoticons:", set(re.findall(SAD_EMO, tweets_text)))

# Palavras mais usadas
fonte: https://www.kaggle.com/youben/twitter-sentiment-analysis

In [ ]:
def most_used_words(text):
    tokens = word_tokenize(text)
    frequency_dist = nltk.FreqDist(tokens)
    print("There is %d different words" % len(set(tokens)))
    return sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)

In [ ]:
most_used_words(data.SentimentText.str.cat())[:100] #100 mais frequentes

In [ ]:
#Mais frequentes sem stopwords
mw = most_used_words(data.SentimentText.str.cat())
most_words = []
for w in mw:
    if len(most_words) == 1000:
        break
    if w in stopwords.words("english"):
        continue
    else:
        most_words.append(w)

# Preparação do texto

In [ ]:
#Limpeza de texto 01
def clean_text_round1(text):
    text = text.lower().strip()
    text = re.sub(r'@\w+\s', ' ', text) #Remover nomes de usuários
    text = re.sub(r'https?://.+\S', ' ', text) #Remover links
    text = re.sub('&[\w*%s*]' % re.escape(string.punctuation), ' ', text) #Apagar '&' e caracteres posteriores 
    text = re.sub(r'#', ' ', text) #Remove hashtags
    text = re.sub('\s{2,}', '', text) #Remover excesso de espaços
    text = re.sub(r'\s', ' ', text) #Remover whitespaces    
    return text

round1 = lambda x: clean_text_round1(x)

In [ ]:
#Criar novo dataframe
clean = pd.DataFrame.from_dict(data.SentimentText.apply(round1)) 

In [ ]:
#Criar coluna com o tamanho de caracteres de cada texto
clean['length'] = [len(text[0]) for text in clean.values]

In [ ]:
clean.describe()
clean

In [ ]:
#Limpeza de texto 02
def clean_text_round2(text):
    #text = re.sub(r'\W', " ", text)
    text = re.sub(r'(.)\1{1,}', r'\1', text) #Remove repetições MELHORAR
    return text

round2 = lambda x : clean_text_round2(x)

In [ ]:
clean.SentimentText = pd.DataFrame.from_dict(clean.SentimentText.apply(round2))
clean

In [ ]:
#Remover linhas com menos de 2 caracteres ou mais de 300
new = np.array([[line[0], int(line[1])] for line in clean.values if 300 > line[1] > 2])
new = pd.DataFrame.from_dict(new)
new.columns = ['text', 'length']

In [ ]:
to_int = lambda x: int(x)
new['length'] = new.length.apply(to_int)

In [ ]:
new.sort_values(by='length', ascending=False)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(new.text.head(10000))
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = new.head(10000).index
data_cv

In [ ]:
print(data_dtm)